# ユニットテストの導入

## 単一モジュールのテスト

In [1]:
!python3 -m unittest booksearch_module.py

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


In [2]:
# -vオプションで詳細な情報を表示
!python3 -m unittest -v booksearch_module.py

test_booksearch (booksearch_module.BookSearchTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


### テスト実行コマンドの簡略化

In [3]:
!cat booksearch_module.py

import unittest

# アプリケーションコード
def booksearch():
    # 任意の処理
    return {}

class BookSearchTest(unittest.TestCase):
    # booksearch()のテストコード
    def test_booksearch(self):
        self.assertEqual({}, booksearch())


if __name__ == '__main__':
    unittest.main()

In [4]:
!python3 booksearch_module.py -v

test_booksearch (__main__.BookSearchTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


## パッケージのテスト

### ディレクトリ構成

In [5]:
# booksearch/、tests/を用意したディレクトリに移動してください
%cd workspace

/Users/suyamar/github/python-practice-book/src/12-unittest/workspace


### サンプルアプリケーションの作成

In [6]:
!cat booksearch/__init__.py

from .core import Book, get_books
__all__ = ['Book', 'get_books']

In [7]:
!cat booksearch/api.py

import json
from urllib import request, parse

def get_json(param):
    with request.urlopen(build_url(param)) as f:
        return json.load(f)

def get_data(url):
    with request.urlopen(url) as f:
        return f.read()

def build_url(param):
    query = parse.urlencode(param)
    return ('https://www.googleapis.com'
            f'/books/v1/volumes?{query}')

In [8]:
!cat booksearch/core.py

import imghdr
import pathlib
from .api import get_data, get_json

class Book:
    """APIレスポンスのVolumeInfo要素に対応"""

    def __init__(self, item):
        self.id = item['id']
        volume_info = item['volumeInfo']
        for k, v in volume_info.items():
            setattr(self, str(k), v)

    def __repr__(self):
        return str(self.__dict__)

    def save_thumbnails(self, prefix):
        """サムネイル画像を保存する"""
        paths = []
        for kind, url in self.imageLinks.items():
            thumbnail = get_data(url)
            # 画像データから拡張子を判定
            ext = imghdr.what(None, h=thumbnail)
            # pathlib.Pathは/演算子でパスを追加できる
            base = pathlib.Path(
              prefix) / f'{self.id}_{kind}'
            filename = base.with_suffix(f'.{ext}')
            filename.write_bytes(thumbnail)
            paths.append(filename)
        return paths

def get_books(q, **params):
    """書籍検索を行う"""
    params['q'] = q
    data = get_json(params)
    return [Book(item) for item in

In [9]:
from booksearch import get_books
books = get_books(q='python')

In [10]:
# 実行時に取得されたデータによって結果は異なる
books[0]

{'id': 'oW63DwAAQBAJ', 'title': '実践力を身につける Pythonの教科書', 'authors': ['クジラ飛行机'], 'publisher': 'マイナビ出版', 'publishedDate': '2016-10-26', 'description': '※この商品はタブレットなど大きいディスプレイを備えた端末で読むことに適しています。また、文字だけを拡大することや、文字列のハイライト、検索、辞書の参照、引用などの機能が使用できません。 便利な簡単プログラムから機械学習までこの1冊で！ 本書はプログラミング言語Python（パイソン）の入門書です。 初めてPythonに取り組む人にとってもわかりやすいように、Pythonの文法の基本を1つずつ丁寧に説明します。小さなプログラムを実際に作りながらの説明なので、その文法がどんなものなのか、どんな時に使えばいいのかを理解しやすくなっています。また、プログラムが動く楽しさを味わいながら進むことができるようになっています。 後半では、より本格的にPythonを使っていくためのプログラミングを学んでいきます。途中少し難しめの解説があるところでは、「後から読んでも大丈夫」というマークを付けて、読み飛ばせるようになっています。 実践編や応用編では、少し長めのプログラムを書いて、デスクトップアプリやWebアプリを作ったり、機械学習で判定をするプログラムを書いたりする作例を紹介しています。これによって、本書で学習したことの理解を深めることもできますし、動くプログラムを作る楽しさや充実感を味わうことができます。 「初心者だけど、とりあえず動くものを作りたい」という目的にも使えますし、少し上達してから、後回しにしていた内容をもう一度読んで、「さらに力を付けて、高度なプログラミングにも挑戦してみたい」という目的にも使える、1冊で2度おいしい本です。', 'industryIdentifiers': [{'type': 'OTHER', 'identifier': 'PKEY:BT000040869800100101900209'}], 'readingModes': {'text': True, 'image': True}, 'pageCount': 232, 'printType': 'BOOK'

# unittestモジュール ── 標準のユニットテストライブラリ

## テストケースの実装

### 前処理、後処理が必要なテストケース

## テストの実行と結果の確認

### テスト失敗時の結果

### テスト失敗時の結果を抑制する

## 特定のテストのみを実行する

### テストケースを直接指定

### テストディスカバリ

# unittest.mockモジュール ── モックオブジェクトの利用

## モックオブジェクトの基本的な使い方

### 任意の値を返す呼び出し可能オブジェクトとして利用する

In [11]:
from unittest.mock import Mock

# 引数で戻り値を設定
mock = Mock(return_value=3)
mock()

3

In [12]:
# return_valueは後からでも設定できる
mock.return_value=4

# 呼び出し時に引数は戻り値に影響しない
mock(1)

4

In [13]:
# 関数では引数がそのまま渡される
mock = Mock(side_effect=lambda x: x % 2)
mock(3)

1

In [14]:
# side_effectは後からでも設定できる
# イテラブルでは呼び出しごとに前から順に返される
mock.side_effect=[2, 1]
mock()

2

In [15]:
mock()

1

In [16]:
# 例外クラスやそのインスタンスではその例外が送出される
mock.side_effect = ValueError('エラーです')
mock()

ValueError: エラーです

### アサーションメソッドで呼び出され方をテストする

In [17]:
mock = Mock(return_value=3)

# まだ一度も呼び出されていないことを確認
mock.assert_not_called()

In [18]:
# 一度だけ呼び出されていることを確認
# まだ一度も呼び出されていないのでエラーになる
mock.assert_called_once()

AssertionError: Expected 'mock' to have been called once. Called 0 times.

In [19]:
# 呼び出してみる
mock(1, a=2)

3

In [20]:
# 呼び出されているのでエラー
mock.assert_not_called()

AssertionError: Expected 'mock' to not have been called. Called 1 times.
Calls: [call(1, a=2)].

In [21]:
# 一度だけ呼び出されていることを確認
mock.assert_called_once()

In [22]:
# 呼び出され方を確認
mock.assert_called_once_with(1, a=2)

In [23]:
mock.assert_called_once_with(1, a=3)

AssertionError: expected call not found.
Expected: mock(1, a=3)
Actual: mock(1, a=2)

In [24]:
# 呼び出し回数は確認せず、一部の引数のみを確認
from unittest.mock import ANY
mock.assert_called_with(1, a=ANY)

## patchを使ったオブジェクトの置き換え

In [25]:
from booksearch import get_books
from unittest.mock import patch

# 対話モードでは__main__モジュールから名前を指定する
with patch('__main__.get_books') as mock_get_books:
    mock_get_books.return_value = []
    print(get_books())

[]


In [26]:
@patch('__main__.get_books')
def test_use_mock(mock_get_books):
    mock_get_books.return_value = []
    return get_books()

In [27]:
test_use_mock()

[]

## mockを利用するテストケースの実例

In [28]:
from booksearch import get_books
book = get_books(q='python')[0]

In [29]:
# 実行時に取得されたデータによって結果は異なる
book.save_thumbnails('tests/data')

[PosixPath('tests/data/oW63DwAAQBAJ_smallThumbnail.jpeg'),
 PosixPath('tests/data/oW63DwAAQBAJ_thumbnail.jpeg')]

# ユースケース別のテストケースの実装

## 環境依存のテストをスキップする

## 例外の発生をテストする

In [30]:
!python3 -m unittest tests.test_core.GetBooksTest -v

test_get_books_no_connection (tests.test_core.GetBooksTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.086s

OK


## 違うパラメータで同じテストを繰り返す

In [31]:
!python3 -m unittest tests.test_api.BuildUrlMultiTest


FAIL: test_build_url_multi (tests.test_api.BuildUrlMultiTest) (q='python', maxResults=1)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/Users/suyamar/github/python-practice-book/src/12-unittest/workspace/tests/test_api.py", line 56, in test_build_url_multi
    self.assertEqual(expected, actual)
AssertionError: 'https://www.googleapis.com/books/v1/volumes?q=python' != 'https://www.googleapis.com/books/v1/volumes?q=python&maxResults=1'
- https://www.googleapis.com/books/v1/volumes?q=python
+ https://www.googleapis.com/books/v1/volumes?q=python&maxResults=1
?                                                     +++++++++++++


FAIL: test_build_url_multi (tests.test_api.BuildUrlMultiTest) (q='python', langRestrict='en')
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/Users/suyamar/github/python-practice-book/src/12-unittest/workspace/tests/test_api.py", lin

## コンテキストマネージャーをテストする

In [32]:
!python3 -m unittest tests.test_api.GetJsonTest -v

test_get_json (tests.test_api.GetJsonTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.057s

OK


# 本章のまとめ